In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

import time

warnings.filterwarnings('ignore')

In [2]:
########################### Vars
#################################################################################
TARGET = 'sales'         # Our main target
END_TRAIN = 1913+28         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [3]:
grid_df = pd.read_pickle('./grid_part_1_update.pkl')
plus_df = pd.read_pickle('./grid_part_3_update.pkl')
grid_df = pd.concat([grid_df, plus_df.drop(["d", "id"], axis=1)], axis=1)
tmp = grid_df[['id','d', 'store_id', 'sales', 'tm_y', 'tm_m', 'tm_d', 'tm_dw', "tm_w"]].copy()

In [4]:
tmp.head()

,id,d,store_id,sales,tm_y,tm_m,tm_d,tm_dw,tm_w
0,HOBBIES_1_008_CA_1_evaluation,1,CA_1,12.0,0,1,29,5,4
1,HOBBIES_1_009_CA_1_evaluation,1,CA_1,2.0,0,1,29,5,4
2,HOBBIES_1_010_CA_1_evaluation,1,CA_1,0.0,0,1,29,5,4
3,HOBBIES_1_012_CA_1_evaluation,1,CA_1,0.0,0,1,29,5,4
4,HOBBIES_1_015_CA_1_evaluation,1,CA_1,4.0,0,1,29,5,4


In [5]:
tmp["tm_w"].describe()

count    4.773540e+07
mean     2.589040e+01
std      1.499768e+01
min      1.000000e+00
25%      1.300000e+01
50%      2.500000e+01
75%      3.900000e+01
max      5.300000e+01
Name: tm_w, dtype: float64

In [6]:
tmp["tm_dw"].describe()

count    4.773540e+07
mean     3.003194e+00
std      2.000796e+00
min      0.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      5.000000e+00
max      6.000000e+00
Name: tm_dw, dtype: float64

## tm_dw=0を集める

In [7]:
tmp2 = tmp[tmp["tm_dw"]==0]

In [9]:
tmp2[tmp2["id"]=="HOBBIES_1_008_CA_1_evaluation"].head()

,id,d,store_id,sales,tm_y,tm_m,tm_d,tm_dw,tm_w
21864,HOBBIES_1_008_CA_1_evaluation,3,CA_1,0.0,0,1,31,0,5
100474,HOBBIES_1_008_CA_1_evaluation,10,CA_1,0.0,0,2,7,0,6
185387,HOBBIES_1_008_CA_1_evaluation,17,CA_1,0.0,0,2,14,0,7
273580,HOBBIES_1_008_CA_1_evaluation,24,CA_1,4.0,0,2,21,0,8
363641,HOBBIES_1_008_CA_1_evaluation,31,CA_1,0.0,0,2,28,0,9


In [10]:
# 24週前までの月曜日のsalesを横付けしていく
for i in range(1, 25):
    print(i)
    tmp2["lag_"+str(i)+"_week"] = tmp2.groupby(['id'])["sales"].transform(lambda x: x.shift(i)).astype(np.float16)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [13]:
tmp2[tmp2["id"]=="HOBBIES_1_008_CA_1_evaluation"].head(6)

,id,d,store_id,sales,tm_y,tm_m,tm_d,tm_dw,tm_w,lag_1_week,...,lag_15_week,lag_16_week,lag_17_week,lag_18_week,lag_19_week,lag_20_week,lag_21_week,lag_22_week,lag_23_week,lag_24_week
21864,HOBBIES_1_008_CA_1_evaluation,3,CA_1,0.0,0,1,31,0,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100474,HOBBIES_1_008_CA_1_evaluation,10,CA_1,0.0,0,2,7,0,6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185387,HOBBIES_1_008_CA_1_evaluation,17,CA_1,0.0,0,2,14,0,7,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273580,HOBBIES_1_008_CA_1_evaluation,24,CA_1,4.0,0,2,21,0,8,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363641,HOBBIES_1_008_CA_1_evaluation,31,CA_1,0.0,0,2,28,0,9,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455475,HOBBIES_1_008_CA_1_evaluation,38,CA_1,0.0,0,3,7,0,10,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
tmp2[tmp2["id"]=="HOBBIES_1_008_CA_1_evaluation"].tail(6)

,id,d,store_id,sales,tm_y,tm_m,tm_d,tm_dw,tm_w,lag_1_week,...,lag_15_week,lag_16_week,lag_17_week,lag_18_week,lag_19_week,lag_20_week,lag_21_week,lag_22_week,lag_23_week,lag_24_week
46454824,HOBBIES_1_008_CA_1_evaluation,1928,CA_1,3.0,5,5,9,0,19,9.0,...,3.0,48.0,17.0,2.0,7.0,8.0,7.0,20.0,27.0,3.0
46668254,HOBBIES_1_008_CA_1_evaluation,1935,CA_1,0.0,5,5,16,0,20,3.0,...,4.0,3.0,48.0,17.0,2.0,7.0,8.0,7.0,20.0,27.0
46881684,HOBBIES_1_008_CA_1_evaluation,1942,CA_1,NaN,5,5,23,0,21,0.0,...,5.0,4.0,3.0,48.0,17.0,2.0,7.0,8.0,7.0,20.0
47095114,HOBBIES_1_008_CA_1_evaluation,1949,CA_1,NaN,5,5,30,0,22,NaN,...,1.0,5.0,4.0,3.0,48.0,17.0,2.0,7.0,8.0,7.0
47308544,HOBBIES_1_008_CA_1_evaluation,1956,CA_1,NaN,5,6,6,0,23,NaN,...,34.0,1.0,5.0,4.0,3.0,48.0,17.0,2.0,7.0,8.0
47521974,HOBBIES_1_008_CA_1_evaluation,1963,CA_1,NaN,5,6,13,0,24,NaN,...,0.0,34.0,1.0,5.0,4.0,3.0,48.0,17.0,2.0,7.0


- これで月曜日(tm_dw=0)に関しては、lag_n_week列を集約すれば移動平均が求められるようになった。
- 注意点として、28日前までの情報は使わないことにしているので、実際はlag_5_week移行の列で平均などを取らないといけない
- 元のデータフレームとは tm_y, tm_w の複合キーが一意にある週を示すので、これとidでmergeすればよい
- ※tm_y：年、tm_w：一年の何週目か(1～53で、多分4/1週が1,3/31週が53)

In [15]:
# 5週前から4週間までさかのぼった時の移動平均
values_5week_4week = [tmp2["lag_"+str(5+i)+"_week"].values for i in range(4)]
tmp2["ave_5week_4week_tm_dw0"] = np.average(values_5week_4week,  axis=0)

In [16]:
# 5週前から8週間までさかのぼった時の移動平均
values_5week_8week = [tmp2["lag_"+str(5+i)+"_week"].values for i in range(8)]
tmp2["ave_5week_8week_tm_dw0"] = np.average(values_5week_8week,  axis=0)

In [17]:
# 5週前から12週間までさかのぼった時の移動平均
values_5week_12week = [tmp2["lag_"+str(5+i)+"_week"].values for i in range(12)]
tmp2["ave_5week_12week_tm_dw0"] = np.average(values_5week_12week,  axis=0)

In [18]:
tmp2

,id,d,store_id,sales,tm_y,tm_m,tm_d,tm_dw,tm_w,lag_1_week,...,lag_18_week,lag_19_week,lag_20_week,lag_21_week,lag_22_week,lag_23_week,lag_24_week,ave_5week_4week_tm_dw0,ave_5week_8week_tm_dw0,ave_5week_12week_tm_dw0
21864,HOBBIES_1_008_CA_1_evaluation,3,CA_1,0.0,0,1,31,0,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21865,HOBBIES_1_009_CA_1_evaluation,3,CA_1,7.0,0,1,31,0,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21866,HOBBIES_1_010_CA_1_evaluation,3,CA_1,1.0,0,1,31,0,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21867,HOBBIES_1_012_CA_1_evaluation,3,CA_1,0.0,0,1,31,0,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21868,HOBBIES_1_015_CA_1_evaluation,3,CA_1,0.0,0,1,31,0,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47552452,FOODS_3_823_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.00,0.250,0.250000
47552453,FOODS_3_824_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.375,0.250000
47552454,FOODS_3_825_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,1.500,1.166992
47552455,FOODS_3_826_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,0.0,1.0,1.0,4.0,8.0,0.0,0.0,1.00,1.125,0.916504


In [19]:
tmp2.to_pickle("tm_dw_0_df_update.pkl")

## 1~6についても移動平均作る

In [20]:
%%time

for j in range(1,7):
    print(j)
    
    tmp2 = tmp[tmp["tm_dw"]==j]
    
    # 24週前までの月曜日のsalesを横付けしていく
    for i in range(1, 25):
        # print(i)
        tmp2["lag_"+str(i)+"_week"] = tmp2.groupby(['id'])["sales"].transform(lambda x: x.shift(i)).astype(np.float16)
        
    # 5週前から4週間までさかのぼった時の移動平均
    values_5week_4week = [tmp2["lag_"+str(5+i)+"_week"].values for i in range(4)]
    tmp2["ave_5week_4week_tm_dw"+str(j)] = np.average(values_5week_4week,  axis=0)
    
    # 5週前から8週間までさかのぼった時の移動平均
    values_5week_8week = [tmp2["lag_"+str(5+i)+"_week"].values for i in range(8)]
    tmp2["ave_5week_8week_tm_dw"+str(j)] = np.average(values_5week_8week,  axis=0)
    
    # 5週前から12週間までさかのぼった時の移動平均
    values_5week_12week = [tmp2["lag_"+str(5+i)+"_week"].values for i in range(12)]
    tmp2["ave_5week_12week_tm_dw"+str(j)] = np.average(values_5week_12week,  axis=0)
    
    tmp2.to_pickle("tm_dw_" + str(j) +"_df.pkl")

1
2
3
4
5
6
CPU times: user 1h 1min 4s, sys: 10min 2s, total: 1h 11min 6s
Wall time: 38min 8s


In [47]:
df = pd.read_pickle("tm_dw_0_df_update.pkl")

In [49]:
df.tail()

,id,d,store_id,sales,tm_y,tm_m,tm_d,tm_dw,tm_w,lag_1_week,...,lag_18_week,lag_19_week,lag_20_week,lag_21_week,lag_22_week,lag_23_week,lag_24_week,ave_5week_4week_tm_dw0,ave_5week_8week_tm_dw0,ave_5week_12week_tm_dw0
47552452,FOODS_3_823_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.00,0.250,0.250000
47552453,FOODS_3_824_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.375,0.250000
47552454,FOODS_3_825_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,1.500,1.166992
47552455,FOODS_3_826_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,0.0,1.0,1.0,4.0,8.0,0.0,0.0,1.00,1.125,0.916504
47552456,FOODS_3_827_WI_3_evaluation,1963,WI_3,NaN,5,6,13,0,24,NaN,...,0.0,0.0,0.0,0.0,8.0,1.0,0.0,0.50,1.625,1.416992


In [41]:
df = df[["id", "tm_y", "tm_w", 'ave_5week_4week_tm_dw0', 'ave_5week_8week_tm_dw0', 'ave_5week_12week_tm_dw0']]

In [42]:
df.tail()

,id,tm_y,tm_w,ave_5week_4week_tm_dw0,ave_5week_8week_tm_dw0,ave_5week_12week_tm_dw0
47552452,FOODS_3_823_WI_3_evaluation,5,24,0.00,0.250,0.250000
47552453,FOODS_3_824_WI_3_evaluation,5,24,0.00,0.375,0.250000
47552454,FOODS_3_825_WI_3_evaluation,5,24,0.75,1.500,1.166992
47552455,FOODS_3_826_WI_3_evaluation,5,24,1.00,1.125,0.916504
47552456,FOODS_3_827_WI_3_evaluation,5,24,0.50,1.625,1.416992


In [43]:
for i in range(1, 7):
    tmp = pd.read_pickle("tm_dw_{}_df.pkl".format(i))
    feat = ["id", "tm_y", "tm_w", 
            'ave_5week_4week_tm_dw'+str(i), 
            'ave_5week_8week_tm_dw'+str(i),
            'ave_5week_12week_tm_dw'+str(i)]
    tmp = tmp[feat]
    df = df.merge(tmp, on=["id", "tm_y", "tm_w"], how="left")
    

In [44]:
df

,id,tm_y,tm_w,ave_5week_4week_tm_dw0,ave_5week_8week_tm_dw0,ave_5week_12week_tm_dw0,ave_5week_4week_tm_dw1,ave_5week_8week_tm_dw1,ave_5week_12week_tm_dw1,ave_5week_4week_tm_dw2,...,ave_5week_12week_tm_dw3,ave_5week_4week_tm_dw4,ave_5week_8week_tm_dw4,ave_5week_12week_tm_dw4,ave_5week_4week_tm_dw5,ave_5week_8week_tm_dw5,ave_5week_12week_tm_dw5,ave_5week_4week_tm_dw6,ave_5week_8week_tm_dw6,ave_5week_12week_tm_dw6
0,HOBBIES_1_008_CA_1_evaluation,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_009_CA_1_evaluation,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_010_CA_1_evaluation,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_012_CA_1_evaluation,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_015_CA_1_evaluation,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6875221,FOODS_3_823_WI_3_evaluation,5,24,0.00,0.250,0.250000,0.50,0.250,0.166626,0.25,...,0.250000,1.0,0.750,0.583496,0.25,0.125,0.250000,1.25,0.625,0.583496
6875222,FOODS_3_824_WI_3_evaluation,5,24,0.00,0.375,0.250000,0.50,0.375,0.250000,0.25,...,0.250000,0.0,0.000,0.000000,0.25,0.375,0.250000,0.00,0.250,0.166626
6875223,FOODS_3_825_WI_3_evaluation,5,24,0.75,1.500,1.166992,0.75,0.500,0.916504,1.25,...,0.583496,0.0,0.500,0.916504,0.75,0.875,1.000000,0.75,0.875,0.833496
6875224,FOODS_3_826_WI_3_evaluation,5,24,1.00,1.125,0.916504,1.00,0.875,1.000000,1.00,...,1.000000,1.5,1.000,0.750000,1.25,1.000,1.000000,1.25,1.625,1.583008


In [46]:
df.to_pickle("week_moving_average.pkl")

In [45]:
tmp.tail()

,id,tm_y,tm_w,ave_5week_4week_tm_dw6,ave_5week_8week_tm_dw6,ave_5week_12week_tm_dw6
47735392,FOODS_3_823_WI_3_evaluation,5,24,1.25,0.625,0.583496
47735393,FOODS_3_824_WI_3_evaluation,5,24,0.00,0.250,0.166626
47735394,FOODS_3_825_WI_3_evaluation,5,24,0.75,0.875,0.833496
47735395,FOODS_3_826_WI_3_evaluation,5,24,1.25,1.625,1.583008
47735396,FOODS_3_827_WI_3_evaluation,5,24,0.50,1.000,1.333008


## 加重平均

In [50]:
weight = [(0.9)**i for i in range(4)]

In [51]:
weight

[1.0, 0.9, 0.81, 0.7290000000000001]

In [53]:
df = pd.read_pickle("tm_dw_0_df_update.pkl")

In [54]:
# 5週前から4週間までさかのぼった時の移動平均
weight = [(0.9)**i for i in range(4)]
values_5week_4week = [df["lag_"+str(5+i)+"_week"].values for i in range(4)]
df["ave_5week_4week_weight_tm_dw0"] = np.average(values_5week_4week,  weights=weight, axis=0)

In [55]:
# 5週前から8週間までさかのぼった時の加重平均
weight = [(0.9)**i for i in range(8)]
values_5week_8week = [df["lag_"+str(5+i)+"_week"].values for i in range(8)]
df["ave_5week_8week_weight_tm_dw0"] = np.average(values_5week_8week,  weights=weight, axis=0)

In [56]:
# 5週前から12週間までさかのぼった時の加重平均
weight = [(0.9)**i for i in range(12)]
values_5week_12week = [df["lag_"+str(5+i)+"_week"].values for i in range(12)]
df["ave_5week_12week_weight_tm_dw0"] = np.average(values_5week_12week,  weights=weight, axis=0)

In [58]:
df[df["id"]=="HOBBIES_1_008_CA_1_evaluation"].tail(6)

,id,d,store_id,sales,tm_y,tm_m,tm_d,tm_dw,tm_w,lag_1_week,...,lag_21_week,lag_22_week,lag_23_week,lag_24_week,ave_5week_4week_tm_dw0,ave_5week_8week_tm_dw0,ave_5week_12week_tm_dw0,ave_5week_4week_weight_tm_dw0,ave_5week_8week_weight_tm_dw0,ave_5week_12week_weight_tm_dw0
46454824,HOBBIES_1_008_CA_1_evaluation,1928,CA_1,3.0,5,5,9,0,19,9.0,...,7.0,20.0,27.0,3.0,1.75,5.250,8.500000,1.688572,4.276188,6.154831
46668254,HOBBIES_1_008_CA_1_evaluation,1935,CA_1,0.0,5,5,16,0,20,3.0,...,8.0,7.0,20.0,27.0,2.25,5.750,4.917969,2.401279,4.650899,4.346905
46881684,HOBBIES_1_008_CA_1_evaluation,1942,CA_1,NaN,5,5,23,0,21,0.0,...,7.0,8.0,7.0,20.0,11.00,6.125,7.750000,12.538529,8.112557,8.950426
47095114,HOBBIES_1_008_CA_1_evaluation,1949,CA_1,NaN,5,5,30,0,22,NaN,...,2.0,7.0,8.0,7.0,15.75,8.500,9.000000,16.809538,10.637369,10.545770
47308544,HOBBIES_1_008_CA_1_evaluation,1956,CA_1,NaN,5,6,6,0,23,NaN,...,17.0,2.0,7.0,8.0,17.50,9.625,9.335938,17.364059,11.153875,10.548630
47521974,HOBBIES_1_008_CA_1_evaluation,1963,CA_1,NaN,5,6,13,0,24,NaN,...,48.0,17.0,2.0,7.0,17.00,9.625,9.500000,15.546089,10.338487,9.872485


In [59]:
df.to_pickle("tm_dw_0_df.pkl")

In [60]:
for k in range(1, 7):
    df = pd.read_pickle("tm_dw_{}_df.pkl".format(k))
    
    # 5週前から4週間までさかのぼった時の移動平均
    weight = [(0.9)**i for i in range(4)]
    values_5week_4week = [df["lag_"+str(5+i)+"_week"].values for i in range(4)]
    df["ave_5week_4week_weight_tm_dw"+str(k)] = np.average(values_5week_4week,  weights=weight, axis=0)
    
    # 5週前から8週間までさかのぼった時の加重平均
    weight = [(0.9)**i for i in range(8)]
    values_5week_8week = [df["lag_"+str(5+i)+"_week"].values for i in range(8)]
    df["ave_5week_8week_weight_tm_dw"+str(k)] = np.average(values_5week_8week,  weights=weight, axis=0)
    
    # 5週前から12週間までさかのぼった時の加重平均
    weight = [(0.9)**i for i in range(12)]
    values_5week_12week = [df["lag_"+str(5+i)+"_week"].values for i in range(12)]
    df["ave_5week_12week_weight_tm_dw"+str(k)] = np.average(values_5week_12week,  weights=weight, axis=0)
    
    df.to_pickle("tm_dw_{}_df.pkl".format(k))

In [61]:
df = pd.read_pickle("tm_dw_0_df.pkl")

In [62]:
df = df[["id", "tm_y", "tm_w", 'ave_5week_4week_weight_tm_dw0',
         'ave_5week_8week_weight_tm_dw0', 'ave_5week_12week_weight_tm_dw0']]

In [63]:
for i in range(1, 7):
    tmp = pd.read_pickle("tm_dw_{}_df.pkl".format(i))
    feat = ["id", "tm_y", "tm_w", 
            'ave_5week_4week_weight_tm_dw'+str(i), 
            'ave_5week_8week_weight_tm_dw'+str(i),
            'ave_5week_12week_weight_tm_dw'+str(i)]
    tmp = tmp[feat]
    df = df.merge(tmp, on=["id", "tm_y", "tm_w"], how="left")

In [ ]:
df.head()

In [65]:
df.to_pickle("week_w_moving_average.pkl")